In [51]:
from datetime import datetime
import os

from langchain.tools import tool  # Create tools with the decorator
from langchain_openai import ChatOpenAI  # Chat model
from langchain.agents import initialize_agent, AgentType  # Agent API
from langchain.memory import ConversationBufferMemory  # Conversational memory

In [52]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

# ══════════════════════════════════════════════════════════════════════════════
# 2. Definir la herramienta personalizada para obtener la fecha actual
# ══════════════════════════════════════════════════════════════════════════════
@tool
def get_current_date(dummy_input: str = "") -> str:
    """
    Devuelve la fecha y hora actuales en formato legible.
    Esta herramienta no recibe argumentos y retorna un string.
    """
    now = datetime.now()
    # Ejemplo: "viernes, 31 de mayo de 2025"
    return now.strftime("%A, %d de %B de %Y")

In [53]:
# ══════════════════════════════════════════════════════════════════════════════
# 3. Inicializar el modelo de lenguaje
# ══════════════════════════════════════════════════════════════════════════════
# Usamos gpt-3.5-turbo con temperatura moderada
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7) 

# ══════════════════════════════════════════════════════════════════════════════
# 4. Configurar memoria para la conversación
# ══════════════════════════════════════════════════════════════════════════════
# ConversationBufferMemory almacena mensajes previos para que el agente conserve contexto.  [oai_citation:3‡LangChain](https://python.langchain.com/docs/versions/v0_3/?utm_source=chatgpt.com) [oai_citation:4‡LangChain Blog](https://blog.langchain.dev/structured-tools/?utm_source=chatgpt.com)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ══════════════════════════════════════════════════════════════════════════════
# 5. Inicializar el agente con la herramienta y la memoria
# ══════════════════════════════════════════════════════════════════════════════
agent = initialize_agent(
    tools=[get_current_date],  # Pasamos la lista de herramientas (en este caso, solo la de fecha)  [oai_citation:5‡GitHub](https://github.com/langchain-ai/langchain/issues/10582?utm_source=chatgpt.com) [oai_citation:6‡LangChain](https://python.langchain.com/docs/how_to/custom_tools/?utm_source=chatgpt.com)
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True  # Muestra en consola la lógica interna del agente
)

In [56]:
# ══════════════════════════════════════════════════════════════════════════════
# 6. Conversación de ejemplo
# ══════════════════════════════════════════════════════════════════════════════
# El agente decidirá, en tiempo de ejecución, si debe llamar a get_current_date.
print("Usuario: Hola, mi nombre es Kike.")
response1 = agent.invoke("Hola, mi nombre es Kike.")
print("Asistente:", response1, "\n")

print("Usuario: ¿Qué día es hoy?")
response2 = agent.invoke("¿Qué día es hoy?")
print("Asistente:", response2, "\n")

# print("Usuario: ¿Recuerdas cómo me llamo?")
# response3 = agent.invoke("¿Recuerdas cómo me llamo?")
# print("Asistente:", response3)

Usuario: Hola, mi nombre es Kike.


> Entering new AgentExecutor chain...
Question: ¿Cómo puedo ayudarte hoy?
Thought: Let's start by getting the current date and time.
Action:
```
{
  "action": "get_current_date",
  "action_input": ""
}
``` 

Observation: Sunday, 01 de June de 2025
Thought:I have the current date and time. Let's continue with the conversation.

Question: ¿Cómo puedo ayudarte hoy?
Thought: I should ask the user for more information or clarify the request.
Final Answer: I am here to help you with anything you need.

> Finished chain.
Asistente: {'input': 'Hola, mi nombre es Kike.', 'chat_history': [HumanMessage(content='Hola, mi nombre es Kike.', additional_kwargs={}, response_metadata={}), AIMessage(content='Sunday, 01 de June de 2025', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hola, mi nombre es Kike.', additional_kwargs={}, response_metadata={}), AIMessage(content='Sunday, 01 de June de 2025', additional_kwargs={}, response_metadata={}), Huma